# Assessing similarities and differences of the different cancer-specifc clusters 


In [ ]:
from anndata import AnnData
from collections import OrderedDict
import anndata as ad
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scanpy as sc
import scipy
import scvi
import seaborn as sns
import time



In [ ]:
folder_path = "/home/withnell/PHD_bcells/Zara_datasets_all_cancer/Datasets_and_scran_after_1_below/newfiguresandfiles/"

# List of tissues to load and process
tissues = ["BRCA", "COAD", "LC", "RCC"]

processed_data_list = []

# Function to add cancer_type metadata for each dataset 
def add_cancer_type(adata, tissue):

    adata.obs['cancer_type'] = tissue
    return adata

for tissue in tissues:
    # Define the input file path
    file_name = f"{tissue}_adata_final2.h5ad"
    adata_filepath = os.path.join(folder_path, file_name)

    if os.path.exists(adata_filepath):
        print(f"Processing file: {adata_filepath}")

        adata = sc.read_h5ad(adata_filepath)

        # Add cancer_type information to the AnnData object
        adata = add_cancer_type(adata, tissue)

        processed_data_list.append(adata)
    else:
        print(f"File not found, skipping: {adata_filepath}")

if processed_data_list:

    combined_adata = processed_data_list[0].concatenate(
        processed_data_list[1:], batch_key="batch", index_unique=None
    )
    print("All datasets successfully concatenated.")
else:
    print("No datasets were loaded.")


In [ ]:
combined_adata.X = combined_adata.layers["counts"].copy()

In [ ]:
combined_adata.obs['patient'] = combined_adata.obs['patient'].astype('category')

In [ ]:
combined_adata.obs['cancerdataid'] = combined_adata.obs['dataid'].astype(str) + "_" + combined_adata.obs['cancer_type'].astype(str)

In [ ]:
combined_adatafullgenes = combined_adata.copy()

In [ ]:
import scanpy as sc

batches = combined_adata.obs['cancerdataid'].unique()

new = {}

# Process each dataset separately
for batch in batches:
    subset_adata = combined_adata[combined_adata.obs['cancerdataid'] == batch].copy()
    sc.pp.normalize_total(subset_adata, target_sum=1e4)
    sc.pp.log1p(subset_adata)
    new[batch] = subset_adata


In [ ]:
combined_adata_new = ad.concat(new, axis=0, join='outer')

In [ ]:

sc.pp.highly_variable_genes(combined_adata_new, batch_key='cancerdataid', n_top_genes = 8000,layer = "counts", flavor="seurat_v3_paper",subset = False

In [ ]:
import scanpy as sc

batches = combined_adata.obs['cancerdataid'].unique()

new = {}

for batch in batches:
    subset_adata = combined_adata[combined_adata.obs['cancerdataid'] == batch].copy()
    sc.pp.scale(subset_adata, max_value=10)

    new[batch] = subset_adata



In [ ]:
combined_adata = ad.concat(new, axis=0, join='outer')

In [ ]:
combined_adata.uns = combined_adata_new.uns
combined_adata.var = combined_adata_new.var


In [ ]:
highly_variable_genes = combined_adata.var['highly_variable']

hv_adata = combined_adata[:, highly_variable_genes].copy()


In [ ]:
scvi.model.SCVI.setup_anndata(
  hv_adata,
  layer="counts",
   categorical_covariate_keys=["dataid", "patient"]
)


In [ ]:
model = scvi.model.SCVI(hv_adata,  n_layers=2, n_latent=30, gene_likelihood="nb")


In [ ]:
model.train(batch_size=130)

In [ ]:
hv_adata.obsm["X_scvi_allcovariates_minustissue"] = model.get_latent_representation()

In [ ]:
hv_adata.X = hv_adata.layers["counts"]
denoised = model.get_normalized_expression(hv_adata, library_size=1e4)
hv_adata.layers["denoised"] = denoised


In [ ]:
hv_adata.obs['SCVI_indiv_tissue'] = hv_adata.obs['leiden_clusters'].astype(str) + "_" + hv_adata.obs['cancer_type'].astype(str)
hv_adata.obs['SCVI_indiv_tissue'].value_counts()
hv_adata.write_h5ad("hv_adata_withalltissues.h5ad")

In [ ]:
hv_adata.X = hv_adata.layers['denoised'].copy()

In [ ]:
# Extract scVI latent space coordinates and cluster labels
latent_coords = hv_adata.obsm['X_scvi_allcovariates_minustissue']
clusters = hv_adata.obs['SCVI_indiv_tissue']

df_latent = pd.DataFrame(latent_coords, columns=[f'latent_{i+1}' for i in range(latent_coords.shape[1])])
df_latent['cluster'] = clusters.values

centroids_latent = df_latent.groupby('cluster').mean().iloc[:, :-1]

distance_matrix_latent = pdist(centroids_latent.values, metric='cosine')
distance_matrix_latent_square = squareform(distance_matrix_latent)

linked = linkage(distance_matrix_latent, method='average')

rcParams['pdf.fonttype'] = 42
plt.figure(figsize=(10, 8))
dendrogram(linked, labels=centroids_latent.index, orientation='top', leaf_rotation=90)
plt.title('Dendrogram of Cluster Centroids (Cosine Similarity)')
plt.xlabel('Clusters')
plt.ylabel('Cosine Distance')
plt.savefig('Dendrogram_of_Cluster_Centroids_fonttype2.pdf')
plt.show()

leaf_order = leaves_list(linked)
ordered_distance_matrix = distance_matrix_latent_square[leaf_order, :][:, leaf_order]
ordered_centroids = centroids_latent.index[leaf_order]

rcParams['pdf.fonttype'] = 42

plt.figure(figsize=(12, 10))
sns.heatmap(ordered_distance_matrix, xticklabels=ordered_centroids, yticklabels=ordered_centroids, cmap='viridis')
plt.title('Cosine Distance Matrix of Cluster Centroids (Reordered)')
plt.savefig('Reordered_Cosine_Distance_Matrix_Heatmap2.pdf')
plt.show()

plt.figure(figsize=(12, 10))

sns.heatmap(
    ordered_distance_matrix,
    xticklabels=ordered_centroids,
    yticklabels=ordered_centroids,
    cmap='coolwarm',
    vmin=0.3,
    vmax=1,
    annot=False,
    fmt='.2f',
    linewidths=0.5
)

rcParams['pdf.fonttype'] = 42

plt.title('Heatmap with Custom Color Cut-Offs')
plt.savefig('Heatmap_with_Color_Cutoffs2.pdf')
plt.show()
